In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from datetime import datetime
from pyspark.sql.functions import udf, col
from pyspark.sql import DataFrame
import re
import time

import os
import sys
import boto3
import boto3.session
import glob
import json
import logging
import pandas as pd
from pyspark.ml.functions import vector_to_array
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField, DateType
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

from pyspark.sql.functions import col, count, udf, min, to_date, max, countDistinct, lit, when
from pyspark.ml.recommendation import ALS
from pyspark.sql import functions as F
from pyspark.sql import DataFrame, Row
from tqdm import tqdm
import re
import time 
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import Normalizer

In [ ]:
name = 'test_ncp'
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.basicConfig(stream=sys.stderr, level=logging.ERROR)
logging.captureWarnings(True)

logging.info('os.getenv: {}'.format(os.getenv))

partition = sys.argv[1]
execution_date = sys.argv[2]
execution_date = datetime.fromisoformat(execution_date)
date_partition = datetime.strftime(execution_date, 'date=%Y-%m-%d/hour=%H')

logging.info("----------------------------------")
logging.info(f"partition: {partition}")
logging.info(f"execution_date: {execution_date}")
logging.info(f"date_partition: {date_partition}")
logging.info("github MikhailYurkov")

dataset = 'messages-sms-current'
data_path = 's3a://oksk/data/abonent_vector/taxonomy_first_layer/{}/*.parquet'.format(date_partition)


In [ ]:
def copyDataFromS3(read_path, write_path, date1 = None, date2 = None, query, rpt):
    if date1 != None:
        start_date = datetime.strptime(date1, "%Y-%m-%d")
        end_date = datetime.strptime(date2, "%Y-%m-%d")
        for date in tqdm(pd.date_range(start=start_date,end=end_date, freq='m')):
            df = spark.read.format("parquet").load(pattern_path.format(date.strftime('%Y-%m-%d')))
            df.createOrReplaceTempView("df")
            df2 = spark.sql(query)
            df2 = df2.repartition(rpt)
            df2.write.mode('append').parquet(write_path)
    else:
        df = spark.read.format("parquet").load(pattern_path))
        df.createOrReplaceTempView("df")
        df2 = spark.sql(query)
        df2 = df2.repartition(rpt)
        df2.write.mode('append').parquet(write_path)
    
    return spark.read.format("parquet").load(write_path.format(date))


In [ ]:
def getDataToQuery(pattern_path):
    df_mas_tax = spark.read.format("parquet").load(pattern_path)

    df_mas_tax.createOrReplaceTempView("df")
    df_mas_tax = spark.sql('''  
                    SELECT * 
                    from df
                    ''')

    ms = df_mas_tax.columns
    ms.remove('phone')
    ms_ch1 = ''
    ms_ch1_sum = ''
    ms_ch1_sum_2 = ''
    ms_ch1_count = ''
    for i in ms:
        ms_ch1 += (',`' + i + '`')
        ms_ch1_sum += (' + `' + i + '`')
        ms_ch1_sum_2 += (',sum(`' + i + '`)')
        ms_ch1_count += (' + count(case when `' + i + '` != 0 then 1 end)')
    return ms_ch1, ms_ch1_sum, ms_ch1_sum_2, ms_ch1_count


In [ ]:
def transformAssembler(df):
    df1p = df.toPandas()
    k1p = df1p.columns
    a = 0
    k1 = []
    b = 0
    k2 = []
    for i in range(0,df1p.shape[1]):
        if df1p.mean()[k1p[i]] < 2*10**(-2):
            k1.append(k1p[i])
            a += 1
        else:
            k2.append(k1p[i])
            b +=1
    k2.pop(0)

    assembler1 = VectorAssembler(
        inputCols=k1,
        outputCol="first_features"
    )
    output1 = assembler1.transform(df1)



    pca1 = PCA(k=5, inputCol="first_features", outputCol="pcaFeatures_old")
    model1 = pca1.fit(output1)
    result1 = model1.transform(output1).select('phone', "pcaFeatures_old")

    assembler1 = VectorAssembler(
        inputCols=k2,
        outputCol="sec_features"
    )
    output1 = assembler1.transform(df1)


    pca1 = PCA(k=15, inputCol="sec_features", outputCol="pcaFeatures_old2")
    model1 = pca1.fit(output1)
    result12 = model1.transform(output1).select('phone', "pcaFeatures_old2")
    result1 = result1.join(result12, on='phone', how='inner')

    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/ncp_11_pca/result1_1'
    result1.write.mode('overwrite').parquet(write_path)
    k1r = []
    for i in result1.columns:
        k1r.append(i)
    k1r.pop(0)

    assembler11 = VectorAssembler(
        inputCols=k1r,
        outputCol="just_features"
    )
    result1 = assembler11.transform(result1)

    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/ncp_11_pca/result1'
    result1.write.mode('overwrite').parquet(write_path)
    return result1
    
    

In [ ]:
if __name__ == "__main__":
    spark = SparkSession.builder.appName('from-s3-to-s3-{}-{}'.format(name, date_partition)).getOrCreate()

    logging.info("Загрузка данных")
    pattern_path = 's3a://oksk/data/abonent_vector/taxonomy_first_layer/date={}'
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_old'

    query = '''  
                         SELECT * 
                         from df
                         '''
    df1 = copyDataFromS3(pattern_path, write_path, date1 = '2022-01-01' , date2 = '2022-07-01' ,query, 15)    
    pattern_path = 's3a://oksk/data/abonent_vector/taxonomy_first_layer/date={}'
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_new'

    query = '''  
                         SELECT * 
                         from df
                         '''
    df2 = copyDataFromS3(pattern_path, write_path, date1 = '2023-01-01' , date2 = '2023-07-01' ,query = query, 15)

    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_old'
    ms_ch1, ms_ch1_sum, ms_ch1_sum_2, ms_ch1_count = getDataToQuery(pattern_path)
    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_new'
    ms_ch2, ms_ch2_sum, ms_ch2_sum_2, ms_ch2_count = getDataToQuery(pattern_path)

    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_old'
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_old_gr_by'

    query = f'''  
                    select phone {ms_ch1_sum_2}
                    from df
                    group by phone
                    '''
    df1 = copyDataFromS3(pattern_path, write_path, query = query, rpt = 10)

    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_new'
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_new_gr_by'

    query = f'''  
                    select phone {ms_ch2_sum_2}
                    from df
                    group by phone
                    '''
    df2 = copyDataFromS3(pattern_path, write_path, query = query, rpt = 10)

    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_old_gr_by'
    ms_ch1, ms_ch1_sum, ms_ch1_sum_2, ms_ch1_count = getDataToQuery(pattern_path)
    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_new_gr_by'
    ms_ch2, ms_ch2_sum, ms_ch2_sum_2, ms_ch2_count = getDataToQuery(pattern_path)

    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_old_gr_by'
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/ncp_11_pca/all_old_gr_by_without_emissions'

    query = f'''
                    select *
                    from df
                    where phone in(
                        select phone
                        from df
                        group by phone
                        having 0 {ms_ch1_count} > 1
                    )
                    '''
    df1 = copyDataFromS3(pattern_path, write_path, query = query, rpt = 10)

    pattern_path = 's3a://oksk/masterdata/target/mike_test/NCP/numbers_change_people/all_new_gr_by'
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/ncp_11_pca/all_new_gr_by_without_emissions'

    query = f'''
                    select *
                    from df
                    where phone in(
                        select phone
                        from df
                        group by phone
                        having 0 {ms_ch2_count} > 1
                    )
                    '''
    df2 = copyDataFromS3(pattern_path, write_path, query = query, rpt = 10)

    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/ncp_11_pca/all_old_gr_by_without_emissions'
    df1 = spark.read.format("parquet").load(write_path)
    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/ncp_11_pca/all_new_gr_by_without_emissions'
    df2 = spark.read.format("parquet").load(write_path)

    result1 = transformAssembler(df1)
    result2 = transformAssembler(df2)


    df_p = result1.join(result2, on='phone', how='inner').select(['phone', "just_features", "just_features2"])

    dot_udf = udf(lambda x, y: float(x.dot(y)), FloatType())
    niz_udf = udf(lambda x, y: float(x.dot(x) ** 0.5 * y.dot(y) ** 0.5), FloatType())

    df_p = df_p.withColumn('cos_taxon', dot_udf(col('just_features'), col('just_features2'))
                        / niz_udf(col('just_features'), col('just_features2')))

    write_path = 's3a://oksk/masterdata/target/mike_test/NCP/target/{}'.format(date_partition)
    df_p.filter('cos_taxon > -0.001 and cos_taxon < 0.001').write.mode('overwrite').parquet(write_path)
    
    logging.info("Запись в хранилище")
    df1.write.mode('overwrite').parquet(write_path)

    spark.stop()